In [ ]:
from zipfile import ZipFile
import pandas as pd
import io

In [ ]:
product_hierarchy = pd.read_csv('/home/tdjiang/github/stored_data/retail_sales_data/product_hierarchy.csv')
store_cities = pd.read_csv('/home/tdjiang/github/stored_data/retail_sales_data/store_cities.csv')

sales_zip = ZipFile('/home/tdjiang/github/stored_data/retail_sales_data/sales.csv.zip', 'r')
sales_data = sales_zip.read('sales.csv')
sales_bytes = io.BytesIO(sales_data)
sales_bytes.seek(0)
sales = pd.read_csv(sales_bytes, converters={'promo_bin_1':str,'promo_bin_2':str,'promo_discount_type_2':str})
df = sales.query("sales != 0 and ~sales.isnull()").reset_index(drop=True)
df['is_promo'] = df[['promo_bin_1','promo_bin_2']].sum(axis=1) != ''

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.seasonal import STL
from causalimpact import CausalImpact
import networkx as nx

def_colours = plt.rcParams['axes.prop_cycle'].by_key()['color']

df_products = pd.read_csv('./data/products.csv')
df_products.columns = map(lambda x: x.lower(), df_products.columns)
df_products = df_products.applymap(lambda x: x.capitalize() if isinstance(x, str) else x)
df_products.set_index('product_key', inplace=True)

fcn_compare = lambda a,b: abs(a-b)/max(a,b)


# Season-Trend decomposition using LOESS.
def decompose_signal(input_signal, period_in_days=14, minimum_heartbeat=0.85):
    sales_decomposition_LOESS = STL(input_signal, period=period_in_days).fit()
    seasonality_flag = sales_decomposition_LOESS.trend > minimum_heartbeat
    return pd.DataFrame({'heartbeat_flag': seasonality_flag,
            'trend': sales_decomposition_LOESS.trend,
            'seasonal': sales_decomposition_LOESS.seasonal,
            'residual': sales_decomposition_LOESS.resid})

def get_taxonomy_from_sku_name_CFAV(sku_name):
    sku_id = int(sku_name.split('_')[0])
    category = df_products.at[sku_id,'product_category_name']
    group = df_products.at[sku_id,'product_group_name']
    return category, group, sku_id


def compare_promo_regular_sales(df_store, df_components, sku_A, idx_holiday_to_exclude, min_promo_days=3, min_regular_days=6):
    '''
        Explain this well as it will go on the paper...
        
        This method splits a CFAV SKU into promotional and regular chunks taking into account the inferred availability (using LOESS)\
        and the holiday periods (any other event can be included in that idx).
        
        The method divides the selling days into sequences of regular and normal days and calculates marginal sales.
    
        'avg_promo_sales' and 'avg_regular_sales' are the sales aggregated across all the slots, whereas
        'slot_promo_avg_sales' and 'slot_promo_avg_sales' represent each slot.
        
        The total sales and total days are not returned, will I need them?
        
        min_promo_days=3, min_regular_days=6 decide the minimum number of days to be taken into consideration for the sequences.
        
            TO-DO: Add the beginning and end of the promotional periods
        
        Updates:
        25.10.2020 - First attempt
    
    '''
    _, _, sku_id_A = get_taxonomy_from_sku_name_CFAV(sku_A)

    sales_sku_A = df_store[f'{sku_id_A}_sales']
    promo_sku_A = df_store[f'{sku_id_A}_is_promotion']
    inferred_availability_sku_A = df_components[f'{sku_id_A}_heartbeat_flag']
    
    analysis_results = []
    
    # only if there are promos
    if promo_sku_A.sum() > 0:

        availability_sku_A = inferred_availability_sku_A & (~idx_holiday_to_exclude)
        availability_value_sku_A = availability_sku_A.sum()/len(availability_sku_A)

        # Split the promotions into slots
        idx_pre_intervention, idx_post_intervention = split_promos_into_sequences(promo_sku_A, \
            min_promo_days=min_promo_days, min_regular_days=min_regular_days)

        num_promo_slots = len(idx_pre_intervention)

        slot_promo_sales = np.zeros(num_promo_slots)
        slot_regular_sales = np.zeros(num_promo_slots)

        slot_promo_days = np.zeros(num_promo_slots)
        slot_regular_days = np.zeros(num_promo_slots)

        for idx_promo_slot in range(0, num_promo_slots):
            idx_pre_intervention_current = idx_pre_intervention[idx_promo_slot]
            idx_post_intervention_current = idx_post_intervention[idx_promo_slot]

            slot_promo_sales[idx_promo_slot] = sales_sku_A[idx_post_intervention_current].sum()
            slot_promo_days[idx_promo_slot] = idx_post_intervention_current.sum()

            slot_regular_sales[idx_promo_slot] = sales_sku_A[idx_pre_intervention_current].sum()
            slot_regular_days[idx_promo_slot] = idx_pre_intervention_current.sum()

        slot_promo_avg_sales = np.divide(slot_promo_sales, slot_promo_days)
        slot_regular_avg_sales = np.divide(slot_regular_sales, slot_regular_days)
        # totals
        total_slot_promo_days = slot_promo_days.sum()
        if total_slot_promo_days>0:
            avg_promo_sales = slot_promo_sales.sum()/total_slot_promo_days
        else:
            avg_promo_sales = 0
        
        total_slot_regular_days = slot_regular_days.sum()
        if total_slot_regular_days>0:
            avg_regular_sales = slot_regular_sales.sum()/total_slot_regular_days
        else:
            avg_regular_sales = 0
            

        # difference between the averages during promo and regular
        difference_averages_promo_to_regular = avg_promo_sales-avg_regular_sales
        # cumulative difference
        cum_difference_sales_promo_to_regular = slot_promo_sales.sum()-slot_regular_sales.sum()
        
        analysis_results.append({'num_promo_slots': num_promo_slots,
        'avg_promo_sales': avg_promo_sales,
        'avg_regular_sales': avg_regular_sales,
        'promo_days': total_slot_promo_days, 
        'regular_days':total_slot_regular_days,
        'difference_averages_promo_to_regular': difference_averages_promo_to_regular,
        'cum_difference_sales_promo_to_regular': cum_difference_sales_promo_to_regular,
        'slot_promo_avg_sales': slot_promo_avg_sales,
        'slot_regular_avg_sales': slot_regular_avg_sales,
        'availability_value_sku_A': availability_value_sku_A})
    
    return analysis_results

def holiday_to_exclude(df_summary:'pd.DataFrame', holidays):
    idx_holidays  = df_summary.index.isin(holidays)
    idx_covid19 = (df_summary.index >= 20210701) & (df_summary.index <= 20211001)
    return idx_holidays | idx_covid19

def max_first_sales_date_shop_AB(df_shops:'pd.DataFrame', shop_code_A, shop_code_B):
    return int(df_shops.query(f'shop_code.isin({[shop_code_A,shop_code_B]})').min_date.max().replace('-',''))

def customers_history_shop(shop_code, folder='./data/customers_per_shop/hcm'):
    df = pd.read_csv(f'{folder}/{shop_code}.csv')
    return df

def customers_by_date_shop_pairs(customers_history_AB:'pd.DataFrame', period_in_days=7):
    df = customers_history_AB\
        .groupby(['date_key','shop_code'], as_index=False)\
        .agg(
            customers=('customer_key','nunique')
        )\
        .pivot(index='date_key',columns='shop_code',values='customers')\
        .fillna(0)\
        .applymap(int)
    return df

def customers_cross_from_victim_shop(customers_history_AB:'pd.DataFrame'):

    shop_code_A, shop_code_B = customers_history_AB.shop_code.unique()

    df = customers_history_AB\
        .assign(
            is_cross_customer = lambda s: (s.date_key > s.groupby('customer_key')['date_key'].transform('min')) & (s.date_key == s.groupby(['customer_key','shop_key'])['date_key'].transform('min'))
        )\
        .query(f"is_cross_customer")\
        .groupby(['date_key','shop_code'], as_index=False)\
        .agg(
            cross_customers=('customer_key','nunique')
        )\
        .pivot(index='date_key', columns='shop_code', values='cross_customers')\
        .rename(columns={shop_code_A:f'{shop_code_A}_cross',shop_code_B:f'{shop_code_B}_cross'})\
        .fillna(0)\
        .applymap(int)
    
    if f'{shop_code_A}_cross' not in df.columns:
        df[f'{shop_code_A}_cross'] = 0
    
    if f'{shop_code_B}_cross' not in df.columns:
        df[f'{shop_code_B}_cross'] = 0
    
    return df

def customers_component_AB(decompostition):
    shop_code_A, shop_code_B = [col for col in customers_by_date_shop_pairs.columns if isinstance(col, int)]
    customers_shop_A = customers_by_date_shop_pairs[shop_code_A]
    df_decomposition_A = decompose_signal()



def compare_cross_regular_customers(df_store:'pd.DataFrame', df_history:'pd.DataFrame', df_components:'pd.DataFrame', shop_code_A, shop_code_B, min_date_key, holidays, min_cross_days=5, min_regular_days=10):

    df_cross = df_history\
        .query(f'shop_code.isin({[shop_code_A,shop_code_B]})')\
        .assign(
            is_cross_customer = lambda s: (s.date_key > s.groupby('customer_key')['date_key'].transform('min')) & (s.date_key == s.groupby(['customer_key','shop_key'])['date_key'].transform('min'))
        )\
        .query(f"is_cross_customer")\
        .groupby(['date_key','shop_code'], as_index=False)\
        .agg(
            cross_customers=('customer_key','nunique')
        )\
        .pivot(index='date_key', columns='shop_code', values='cross_customers')\
        .reset_index()\
        .rename(columns={shop_code_A:f'{shop_code_A}_cross',shop_code_B:f'{shop_code_B}_cross'})\
        .fillna(0)\
        .applymap(int)
    
    df_summary = df_store[['date_key',shop_code_A,shop_code_B]]\
        .merge(df_cross, on='date_key', how='left')\
        .fillna(0)\
        .applymap(int)\
        .query(f'date_key >= {min_date_key}')
    df_summary[[f'{shop_code_A}_iscross',f'{shop_code_B}_iscross']] = df_summary[[f'{shop_code_A}_cross',f'{shop_code_B}_cross']].applymap(lambda x: False if x==0 else True)

    customers_shop_A = df_summary[shop_code_A]
    cross_shop_A = df_summary[f'{shop_code_A}_iscross']
    inferred_availability_shop_A = df_components.query(f'date_key >= {min_date_key}')[f'{shop_code_A}_heartbeat_flag']
    
    analysis_results = []
    idx_holiday_to_exclude = holiday_to_exclude(df_summary, holidays)
    
    # only if there are cross shops
    if cross_shop_A.sum() > 0:

        availability_shop_A = inferred_availability_shop_A & (~idx_holiday_to_exclude)
        availability_value_shop_A = availability_shop_A.sum()/len(availability_shop_A)

        # Split the cross shops into slots
        idx_pre_intervention, idx_post_intervention = split_cross_into_sequences(cross_shop_A, \
            min_cross_days=min_cross_days, min_regular_days=min_regular_days)

        num_cross_slots = len(idx_pre_intervention)

        slot_cross_customers = np.zeros(num_cross_slots)
        slot_regular_customers = np.zeros(num_cross_slots)

        slot_cross_days = np.zeros(num_cross_slots)
        slot_regular_days = np.zeros(num_cross_slots)

        for idx_cross_slot in range(0, num_cross_slots):
            idx_pre_intervention_current = idx_pre_intervention[idx_cross_slot]
            idx_post_intervention_current = idx_post_intervention[idx_cross_slot]

            slot_cross_customers[idx_cross_slot] = customers_shop_A[idx_post_intervention_current].sum()
            slot_cross_days[idx_cross_slot] = idx_post_intervention_current.sum()

            slot_regular_customers[idx_cross_slot] = customers_shop_A[idx_pre_intervention_current].sum()
            slot_regular_days[idx_cross_slot] = idx_pre_intervention_current.sum()

        slot_cross_avg_customers = np.divide(slot_cross_customers, slot_cross_days)
        slot_regular_avg_customers = np.divide(slot_regular_customers, slot_regular_days)
        # totals
        total_slot_cross_days = slot_cross_days.sum()
        if total_slot_cross_days>0:
            avg_cross_customers = slot_cross_customers.sum()/total_slot_cross_days
        else:
            avg_cross_customers = 0
        
        total_slot_regular_days = slot_regular_days.sum()
        if total_slot_regular_days>0:
            avg_regular_customers = slot_regular_customers.sum()/total_slot_regular_days
        else:
            avg_regular_customers = 0
            

        # difference between the averages during cross shops and regular
        difference_averages_cross_to_regular = avg_cross_customers-avg_regular_customers
        # cumulative difference
        cum_difference_customers_cross_to_regular = slot_cross_customers.sum()-slot_regular_customers.sum()
        
        analysis_results.append({
            'shop_code_A':shop_code_A,
            'shop_code_B':shop_code_B,
            'num_cross_slots': num_cross_slots,
            'avg_cross_customers': avg_cross_customers,
            'avg_regular_customers': avg_regular_customers,
            'cross_days': total_slot_cross_days, 
            'regular_days':total_slot_regular_days,
            'difference_averages_cross_to_regular': difference_averages_cross_to_regular,
            'cum_difference_customers_cross_to_regular': cum_difference_customers_cross_to_regular,
            'slot_promo_avg_customers': slot_cross_avg_customers,
            'slot_regular_avg_customers': slot_regular_avg_customers,
            'availability_value_shop_A': availability_value_shop_A
        })
    
    return analysis_results


def calculate_causal_impact_with_covariates(sku_id_A, promo_sku_A, availability_sku_A, sales_sku_A, \
    sku_id_B, promo_sku_B, availability_sku_B, df_sales_covariates, \
    idx_pre_intervention, idx_post_intervention, \
    idx_holiday_to_exclude,
    min_diff_in_units_from_reg_to_promo, 
    min_ratio_change = 0.3,
    do_exclude_promos_SKU_B = True, be_verbose=True, \
    min_overlapping_days_regular=5, min_overlapping_days_promo=3):
    '''
        This is the method used to populate the paper results
    '''

    # use this flag to exclude sku_B if on promo
    total_days = promo_sku_A.shape[0]
    num_days = np.arange(total_days)
    combined_availability = availability_sku_A & availability_sku_B & (~idx_holiday_to_exclude)

    causal_analysis = []
    
    sales_sku_B = df_sales_covariates.iloc[:,0]
    
    total_slots = len(idx_pre_intervention)

    for idx_promo_slot in range(0, total_slots):

        idx_pre_intervention_current = idx_pre_intervention[idx_promo_slot]
        idx_post_intervention_current = idx_post_intervention[idx_promo_slot]

        # # #
        # promo days == 'post-intervention'
        # # #
        idx_overlapping_days_promo = combined_availability & idx_post_intervention_current
        total_overlapping_days_promo = idx_overlapping_days_promo.sum()


        # overlapping promo days. Both SKUs on promo, "competing promos"
        idx_competing_promo_days = idx_overlapping_days_promo & promo_sku_B
        competing_promo_days = idx_competing_promo_days.sum()


        # # #
        # regular days == 'pre-intervention'
        # # #
        # A period should not be marked as 'regular' if SKU_B is on promotion
        if do_exclude_promos_SKU_B:
            idx_overlapping_days_regular = combined_availability & idx_pre_intervention_current & (~promo_sku_B)
        else:
            idx_overlapping_days_regular = combined_availability & idx_pre_intervention_current
        total_overlapping_days_regular = idx_overlapping_days_regular.sum()
        
        # Minimum requirements of overlap. Otherwise the analysis does not make much sense.
        # numerical index (for Causal Impact)
        if (total_overlapping_days_regular>=min_overlapping_days_regular) & \
            (total_overlapping_days_promo>=min_overlapping_days_promo):
            
            ind_regular_days = num_days[idx_overlapping_days_regular]
            start_regular = ind_regular_days.min()
            end_regular = ind_regular_days.max()
            
            ind_promo_days = num_days[idx_overlapping_days_promo]
            start_promo = ind_promo_days.min()
            end_promo = ind_promo_days.max()

            # sales of SKU_B
            # during regular
            sku_B_regular_avg_sales = sales_sku_B[idx_overlapping_days_regular].mean()
            # during promo
            sku_B_avg_sales_during_promo_sku_A = sales_sku_B[idx_overlapping_days_promo].mean()
            # Difference in average sales between the regular and the promotional one.
            diff_in_units_from_reg_to_promo = sku_B_regular_avg_sales-sku_B_avg_sales_during_promo_sku_A
            
            # can we review the post promotional sales?
            post_period_start = end_promo+1
            post_period_end = np.min([end_promo+8, total_days])
            post_promo_days = post_period_end-post_period_start+1
            sku_B_regular_post_promo_avg_sales = sales_sku_B[post_period_start:post_period_end].mean()
            # post promo should be larger than during the cannibalisation
            diff_in_units_from_promo_to_pos_promo = sku_B_avg_sales_during_promo_sku_A-sku_B_regular_post_promo_avg_sales
            
            post_promo_flag = (-diff_in_units_from_promo_to_pos_promo > min_diff_in_units_from_reg_to_promo*0.25)
            
            '''
            if idx_promo_slot+1 < total_slots:
                # during regular
                idx_reg_post_intervention = idx_pre_intervention[idx_promo_slot+1]
                sku_B_regular_post_promo_avg_sales = sales_sku_B[idx_reg_post_intervention].mean()
                # post promo should be larger than during the cannibalisation
                diff_in_units_from_promo_to_pos_promo = sku_B_avg_sales_during_promo_sku_A-sku_B_regular_post_promo_avg_sales
                post_promo_flag = (-diff_in_units_from_promo_to_pos_promo>min_diff_in_units_from_reg_to_promo)
            else:
                diff_in_units_from_promo_to_pos_promo = np.nan
                post_promo_flag = True
            '''

            # delta
            ratio_change = fcn_compare(sku_B_avg_sales_during_promo_sku_A, sku_B_regular_avg_sales)
            if be_verbose:
                print(f'Summary of the current scenario (slot {idx_promo_slot})')
                print(f'Before SKU A going on promo, the SKUs overlap for {total_overlapping_days_regular} days (promos on sku_B excluded: {do_exclude_promos_SKU_B})')
                if end_regular+1<start_promo:
                    print(f'There is a gap of {start_promo-end_regular} days between the regular days and the beginning of the promotion (due to availability/promotional period of SKU_B)')
                print(f'When SKU_A is on promo, the SKUs overlap for {total_overlapping_days_promo} days')
                print(f'During the overlapping days, SKU B is on promo for {competing_promo_days} days')

                print(f'Average sales of sku B before sku A on promo {sku_B_regular_avg_sales:.2f}')
                print(f'Average sales of sku B during sku A on promo {sku_B_avg_sales_during_promo_sku_A:.2f}')
                print(f'Average sales of sku B after sku A on promo {sku_B_regular_post_promo_avg_sales:.2f} over {post_promo_days} days - {post_promo_flag}')
                print(f'Diff in units from regular to promotion {-diff_in_units_from_reg_to_promo:.2f}')
                print(f'Diff in units from cannibalisation to regular {-diff_in_units_from_promo_to_pos_promo:.2f}')
                
                print(f'Ratio of change {ratio_change:3.2f} (the lower the closer (0,1)) {ratio_change>min_ratio_change} \n')


            if (ratio_change>min_ratio_change) & (diff_in_units_from_reg_to_promo > min_diff_in_units_from_reg_to_promo) & post_promo_flag:

                idx_regular_days = np.array([start_regular, end_regular]).tolist()
                idx_promo_days   = np.array([start_promo, end_promo]).tolist()

                print('Running Causal Impact...')
                ci = CausalImpact(df_sales_covariates, idx_regular_days, idx_promo_days)
                '''
                https://github.com/dafiti/causalimpact/blob/8d881fc5c270348d8c8ff59c936997a75d7c5fac/causalimpact/main.py#L88
                
                First column must contain the `y` measured value 
                while the others contain the covariates `X` that are used in the 
                linear regression component of the model.
                '''
                #ci.lower_upper_percentile
                avg_actual = ci.summary_data.loc['actual', 'average']
                # Had the promo not been launched on the cannibal, we would have sold this amount.
                avg_predicted = ci.summary_data.loc['predicted', 'average']
                avg_abs_effect = ci.summary_data.loc['abs_effect', 'average']
                # This can be seen as the number of units that the cannibal is taking from the victim
                cum_abs_effect = ci.summary_data.loc['abs_effect', 'cumulative']
                posterior_tail_prob = ci.p_value
                prob_causal_effect = (1-ci.p_value)*100
                print(f'CausalImpact >> Probability of a causal event {prob_causal_effect:.2f}')

                temp_dict = {'cannibal':sku_id_A,
                            'victim':sku_id_B,
                            'slot_number': idx_promo_slot,
                            'idx_regular_days': idx_regular_days,
                            'idx_promo_days': idx_promo_days,
                            'total_overlapping_days_regular': total_overlapping_days_regular,
                            'regular_to_promo_gap': start_promo-end_regular-1,
                            'total_overlapping_days_promo': total_overlapping_days_promo,
                            'competing_promo_days': competing_promo_days,
                            'sku_B_regular_avg_sales': sku_B_regular_avg_sales,
                            'sku_B_avg_sales_during_promo_sku_A': sku_B_avg_sales_during_promo_sku_A,
                            'diff_in_units_from_reg_to_promo': diff_in_units_from_reg_to_promo,
                            'diff_in_units_from_promo_to_pos_promo': diff_in_units_from_promo_to_pos_promo,
                            'ratio_change': ratio_change,
                            'avg_actual':avg_actual,
                            'avg_predicted': avg_predicted,
                            'avg_abs_effect': avg_abs_effect,
                            'cum_abs_effect': cum_abs_effect,
                            'posterior_tail_prob': posterior_tail_prob,
                            'prob_causal_effect': prob_causal_effect
                            }
                causal_analysis.append(temp_dict)
                
    return causal_analysis

def add_graph_relationship(node_A, node_B, edge_properties: dict):
    DG = nx.DiGraph()

    DG.add_node(node_A['name'], **node_A['properties'])

    d = dict()
    DG.add_node(node_B['name'], **node_B['properties'])

    edge_label = '\n'.join([f'{k}: {v:3.2f}' for k,v in edge_properties.items()])  
    DG.add_edge(node_A['name'], node_B['name'], **edge_properties, label=edge_label)

        
def split_promos_into_sequences(idx_promos: 'pd.Series', min_promo_days=4, min_regular_days=6):
    '''
    Group the indices of a promotion into sequences of pre and post promotion
    '''
    # Groups/sequences
    seqs = (idx_promos.shift(1)!=idx_promos).cumsum()
    promo_seqs = seqs[idx_promos]
    # Indices
    idx_pre_intervention = []
    idx_post_intervention = []
    for value_promo_seqs in promo_seqs.unique():
        idx_current_promo = seqs==value_promo_seqs
        prev_seq = value_promo_seqs-1
        idx_current_regular = seqs==prev_seq
        current_promo_length = idx_current_promo.sum()
        current_regular_length = idx_current_regular.sum()
        if (current_promo_length >= min_promo_days) and (current_regular_length >= min_regular_days):
            idx_pre_intervention.append(idx_current_regular)
            idx_post_intervention.append(idx_current_promo)
    return idx_pre_intervention, idx_post_intervention

def split_cross_into_sequences(idx_cross: 'pd.Series', min_cross_days=5, min_regular_days=10):
    # Groups/sequences
    seqs = (idx_cross.shift(1)!=idx_cross).cumsum()
    promo_seqs = seqs[idx_cross]
    # Indices
    idx_pre_intervention = []
    idx_post_intervention = []
    for value_promo_seqs in promo_seqs.unique():
        idx_current_promo = seqs==value_promo_seqs
        prev_seq = value_promo_seqs-1
        idx_current_regular = seqs==prev_seq
        current_promo_length = idx_current_promo.sum()
        current_regular_length = idx_current_regular.sum()
        if (current_promo_length >= min_cross_days) and (current_regular_length >= min_regular_days):
            idx_pre_intervention.append(idx_current_regular)
            idx_post_intervention.append(idx_current_promo)
    return idx_pre_intervention, idx_post_intervention
